<a href="https://colab.research.google.com/github/ShuYangConlany/Enefit/blob/main/Enefit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install optuna
!pip install lightgbm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.8 MB/s eta 0:00:00


In [ ]:
import os
import gc
import pickle
import time

import numpy as np
import pandas as pd
import polars as pl

from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import mean_absolute_error
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import VotingRegressor

from joblib import dump
from joblib import load

import lightgbm as lgb

import optuna
import matplotlib.pyplot as plt

In [ ]:
def feature_eng(df_data, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target, working_days):
    working_days = (
        working_days
        .with_columns(
            pl.col("date").cast(pl.Date)
        )
    )

    df_data = (
        df_data
        .with_columns(
            pl.col("datetime").cast(pl.Date).alias("date"),
        )
    )

    df_client = (
        df_client
        .with_columns(
            (pl.col("date") + pl.duration(days=2)).cast(pl.Date)
        )
    )

    df_gas = (
        df_gas
        .rename({"forecast_date": "date"})
        .with_columns(
            (pl.col("date") + pl.duration(days=1)).cast(pl.Date)
        )
    )

    df_electricity = (
        df_electricity
        .rename({"forecast_date": "datetime"})
        .with_columns(
            pl.col("datetime") + pl.duration(days=1)
        )
    )

    df_location = (
        df_location
        .with_columns(
            pl.col("latitude").cast(pl.datatypes.Float32),
            pl.col("longitude").cast(pl.datatypes.Float32)
        )
    )

    df_forecast = (
        df_forecast
        .rename({"forecast_datetime": "datetime"})
        .with_columns(
            pl.col("latitude").cast(pl.datatypes.Float32),
            pl.col("longitude").cast(pl.datatypes.Float32),
            #pl.col('datetime').dt.convert_time_zone("Europe/Bucharest").dt.replace_time_zone(None).cast(pl.Datetime("us")),
            pl.col('datetime').dt.replace_time_zone(None).cast(pl.Datetime("us"))
            #pl.col('datetime').cast(pl.Datetime)
        )
        .join(df_location, how="left", on=["longitude", "latitude"])
        .drop("longitude", "latitude")
    )

    df_historical = (
        df_historical
        .with_columns(
            pl.col("latitude").cast(pl.datatypes.Float32),
            pl.col("longitude").cast(pl.datatypes.Float32),
            pl.col("datetime") + pl.duration(hours=37)
        )
        .join(df_location, how="left", on=["longitude", "latitude"])
        .drop("longitude", "latitude")
    )

    df_forecast_date = (
        df_forecast
        .groupby("datetime").mean()
        .drop("county")
    )

    df_forecast_local = (
        df_forecast
        .filter(pl.col("county").is_not_null())
        .groupby("county", "datetime").mean()
    )

    df_historical_date = (
        df_historical
        .groupby("datetime").mean()
        .drop("county")
    )

    df_historical_local = (
        df_historical
        .filter(pl.col("county").is_not_null())
        .groupby("county", "datetime").mean()
    )
    df_data = (
        df_data
        .join(df_gas, on="date", how="left")
        .join(df_client, on=["county", "is_business", "product_type", "date"], how="left")
        .join(df_electricity, on="datetime", how="left")

        .join(df_forecast_date, on="datetime", how="left", suffix="_fd")
        .join(df_forecast_local, on=["county", "datetime"], how="left", suffix="_fl")
        .join(df_historical_date, on="datetime", how="left", suffix="_hd")
        .join(df_historical_local, on=["county", "datetime"], how="left", suffix="_hl")

        .join(df_forecast_date.with_columns(pl.col("datetime") + pl.duration(days=7)), on="datetime", how="left", suffix="_fdw")
        .join(df_forecast_local.with_columns(pl.col("datetime") + pl.duration(days=7)), on=["county", "datetime"], how="left", suffix="_flw")
        .join(df_historical_date.with_columns(pl.col("datetime") + pl.duration(days=7)), on="datetime", how="left", suffix="_hdw")
        .join(df_historical_local.with_columns(pl.col("datetime") + pl.duration(days=7)), on=["county", "datetime"], how="left", suffix="_hlw")

        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=2)).rename({"target": "target_1"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=3)).rename({"target": "target_2"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=4)).rename({"target": "target_3"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=5)).rename({"target": "target_4"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=6)).rename({"target": "target_5"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=7)).rename({"target": "target_6"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=8)).rename({"target": "target_7"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=9)).rename({"target": "target_8"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=10)).rename({"target": "target_9"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=11)).rename({"target": "target_10"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=12)).rename({"target": "target_11"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=13)).rename({"target": "target_12"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=14)).rename({"target": "target_13"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")

        .join(working_days, on="date", how="left")
        .with_columns(
            pl.col("datetime").dt.ordinal_day().alias("dayofyear"),
            pl.col("datetime").dt.hour().alias("hour"),
            pl.col("datetime").dt.day().alias("day"),
            pl.col("datetime").dt.weekday().alias("weekday"),
            pl.col("datetime").dt.month().alias("month"),
            pl.col("datetime").dt.year().alias("year"),
        )
        .with_columns(
            pl.concat_str("county", "is_business", "product_type", "is_consumption", separator="_").alias("category_1"),
        )

        .with_columns(
            (np.pi * pl.col("dayofyear") / 183).sin().alias("sin(dayofyear)"),
            (np.pi * pl.col("dayofyear") / 183).cos().alias("cos(dayofyear)"),
            (np.pi * pl.col("hour") / 12).sin().alias("sin(hour)"),
            (np.pi * pl.col("hour") / 12).cos().alias("cos(hour)"),
        )

        .with_columns(
            pl.col(pl.Float64).cast(pl.Float32),
        )
        .drop("date", "hour", "dayofyear")
    )

    # return df_data, df_historical_local
    return df_data

In [ ]:
def to_pandas(X, y=None):
    cat_cols = ["county", "is_business", "product_type", "is_consumption", "category_1"]

    if y is not None:
        df = pd.concat([X.to_pandas(), y.to_pandas()], axis=1)
    else:
        df = X.to_pandas()

    df = df.merge(df.groupby(['category_1', 'year', 'month', 'day'])['target_1'].sum().reset_index(),
                  on=['category_1', 'year', 'month', 'day'],
                  how='left',
                  suffixes=['', '_sum'])

    df = df.set_index("row_id")
    df[cat_cols] = df[cat_cols].astype("category")

    df["target_mean"] = df[[f"target_{i}" for i in range(1, 7)]].mean(1)
    df["target_std"] = df[[f"target_{i}" for i in range(1, 7)]].std(1)
    df["target_ratio"] = df["target_6"] / (df["target_7"] + 1e-3)

    return df

In [ ]:
root = '/content/drive/My Drive/Kaggle-Enefit'
data_cols        = ['target', 'county', 'is_business', 'product_type', 'is_consumption', 'datetime', 'row_id']
df_data_cols     = ['county', 'is_business', 'product_type', 'target', 'is_consumption', 'datetime', 'row_id']
client_cols      = ['product_type', 'county', 'eic_count', 'installed_capacity', 'is_business', 'date']
gas_cols         = ['forecast_date', 'lowest_price_per_mwh', 'highest_price_per_mwh']
electricity_cols = ['forecast_date', 'euros_per_mwh']
forecast_cols    = ['latitude', 'longitude', 'hours_ahead', 'temperature', 'dewpoint', 'cloudcover_high', 'cloudcover_low', 'cloudcover_mid', 'cloudcover_total', '10_metre_u_wind_component', '10_metre_v_wind_component', 'forecast_datetime', 'direct_solar_radiation', 'surface_solar_radiation_downwards', 'snowfall', 'total_precipitation']
historical_cols  = ['datetime', 'temperature', 'dewpoint', 'rain', 'snowfall', 'surface_pressure','cloudcover_total','cloudcover_low','cloudcover_mid','cloudcover_high','windspeed_10m','winddirection_10m','shortwave_radiation','direct_solar_radiation','diffuse_radiation','latitude','longitude']
location_cols    = ['longitude', 'latitude', 'county']
target_cols      = ['target', 'county', 'is_business', 'product_type', 'is_consumption', 'datetime']


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%time
df_data        = pl.read_csv(os.path.join(root, "train.csv"), columns=data_cols, try_parse_dates=True)
df_client      = pl.read_csv(os.path.join(root, "client.csv"), columns=client_cols, try_parse_dates=True)
df_gas         = pl.read_csv(os.path.join(root, "gas_prices.csv"), columns=gas_cols, try_parse_dates=True)
df_electricity = pl.read_csv(os.path.join(root, "electricity_prices.csv"), columns=electricity_cols, try_parse_dates=True)
df_forecast    = pl.read_csv(os.path.join(root, "forecast_weather.csv"), columns=forecast_cols, try_parse_dates=True)
df_historical  = pl.read_csv(os.path.join(root, "historical_weather.csv"), columns=historical_cols, try_parse_dates=True)
df_location    = pl.read_csv(os.path.join(root,"Other-Data/county_lon_lats.csv"), columns=location_cols, try_parse_dates=True)
df_target      = df_data.select(target_cols)
working_days   = pl.read_csv(os.path.join(root,'Other-Data/working_days.csv'), try_parse_dates=True)

schema_data        = df_data.schema
schema_client      = df_client.schema
schema_gas         = df_gas.schema
schema_electricity = df_electricity.schema
schema_forecast    = df_forecast.schema
schema_historical  = df_historical.schema
schema_target      = df_target.schema

CPU times: user 7.73 s, sys: 1.69 s, total: 9.43 s
Wall time: 27.7 s


In [ ]:
X, y = df_data.drop("target"), df_data.select("target")

X = feature_eng(X, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target, working_days)

df_train = to_pandas(X, y)

df_train.fillna(-999, inplace=True)

In [ ]:
df_train = df_train[df_train["target"].notnull()]
df_train

,county,is_business,product_type,is_consumption,datetime,lowest_price_per_mwh,highest_price_per_mwh,eic_count,installed_capacity,euros_per_mwh,...,category_1,sin(dayofyear),cos(dayofyear),sin(hour),cos(hour),target,target_1_sum,target_mean,target_std,target_ratio
row_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,0,2021-09-01 00:00:00,-999.0,-999.0,-999.0,-999.000000,-999.00,...,0_0_1_0,-0.866025,-0.500000,0.000000,1.000000,0.713,0.000000,-999.000000,-999.000000,-999.000000
1,0,0,1,1,2021-09-01 00:00:00,-999.0,-999.0,-999.0,-999.000000,-999.00,...,0_0_1_1,-0.866025,-0.500000,0.000000,1.000000,96.590,0.000000,-999.000000,-999.000000,-999.000000
2,0,0,2,0,2021-09-01 00:00:00,-999.0,-999.0,-999.0,-999.000000,-999.00,...,0_0_2_0,-0.866025,-0.500000,0.000000,1.000000,0.000,0.000000,-999.000000,-999.000000,-999.000000
3,0,0,2,1,2021-09-01 00:00:00,-999.0,-999.0,-999.0,-999.000000,-999.00,...,0_0_2_1,-0.866025,-0.500000,0.000000,1.000000,17.314,0.000000,-999.000000,-999.000000,-999.000000
4,0,0,3,0,2021-09-01 00:00:00,-999.0,-999.0,-999.0,-999.000000,-999.00,...,0_0_3_0,-0.866025,-0.500000,0.000000,1.000000,2.904,0.000000,-999.000000,-999.000000,-999.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018347,15,1,0,1,2023-05-31 23:00:00,29.0,34.0,15.0,620.000000,-1.29,...,15_1_0_1,0.522133,-0.852864,-0.258819,0.965926,197.233,11490.608398,250.953674,106.506004,2.278837
2018348,15,1,1,0,2023-05-31 23:00:00,29.0,34.0,20.0,624.500000,-1.29,...,15_1_1_0,0.522133,-0.852864,-0.258819,0.965926,0.000,2889.723877,0.000000,0.000000,0.000000
2018349,15,1,1,1,2023-05-31 23:00:00,29.0,34.0,20.0,624.500000,-1.29,...,15_1_1_1,0.522133,-0.852864,-0.258819,0.965926,28.404,810.093018,34.342167,2.976434,0.880700


In [ ]:
null_value_stats = df_train.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]
print(df_train.dtypes)

county                  category
is_business             category
product_type            category
is_consumption          category
datetime          datetime64[ns]
                       ...      
target                   float64
target_1_sum             float32
target_mean              float32
target_std               float32
target_ratio             float32
Length: 146, dtype: object


In [ ]:
!pip install catboost
from catboost import CatBoostRegressor, Pool, metrics, cv
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 16.7 MB/s eta 0:00:00


In [ ]:
model = CatBoostRegressor(
    task_type='GPU',
    iterations=1000,
    learning_rate=0.1,
    depth=10,
    random_seed=42,
    logging_level='Silent'
)

In [ ]:
X = df_train.drop(['target', 'datetime'], axis=1)
y = df_train['target']
batch_size = 2018350
num_batches = len(X) // batch_size
# X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.75, random_state=42)

categorical_features_indices = [ 'county', 'is_business','product_type','is_consumption','category_1']
#,'hours_ahead','hours_ahead_fl','hours_ahead_fdw','hours_ahead_flw','is_working','day','weekday','month','year','category_1']
# for i in range(1):
#   start_idx = i * batch_size
#   end_idx = start_idx + batch_size
#   X_batch = X.iloc[start_idx:end_idx]
#   y_batch = y.iloc[start_idx:end_idx]
#   X_train, X_validation, y_train, y_validation = train_test_split(X_batch, y_batch, train_size=0.8, random_state=42)

#   model.fit(
#     X_train, y_train,
#     cat_features=categorical_features_indices,
#     eval_set=(X_validation, y_validation),
#     plot=True
#   )

#   # model.fit(train_pool, plot=True, eval_set=val_pool, verbose=500)
# for i in range(2):
i=0
start_idx = i * batch_size
end_idx = start_idx + batch_size
X_batch = X.iloc[start_idx:end_idx]
y_batch = y.iloc[start_idx:end_idx]
X_train, X_validation, y_train, y_validation = train_test_split(X_batch, y_batch, train_size=0.8, random_state=42)

model.fit(
    X_train, y_train,
    cat_features=categorical_features_indices,
    eval_set=(X_validation, y_validation),
    plot=True,
        # init_model=model if i > 0 else None
)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [ ]:
mean_absolute_error(df_train['target'], model.predict(df_train.drop(['target', 'datetime'], axis=1)))

25.177782905625307

In [ ]:
import enefit

env = enefit.make_env()
iter_test = env.iter_test()

Exception: You can only call `make_env()` once.

In [ ]:
for (test, revealed_targets, client, historical_weather,
        forecast_weather, electricity_prices, gas_prices, sample_prediction) in iter_test:
    test = test.rename(columns={"prediction_datetime": "datetime"})

    df_test            = pl.from_pandas(test[data_cols[1:]], schema_overrides=schema_data)
    df_new_client      = pl.from_pandas(client[client_cols], schema_overrides=schema_client)
    df_new_gas         = pl.from_pandas(gas_prices[gas_cols], schema_overrides=schema_gas)
    df_new_electricity = pl.from_pandas(electricity_prices[electricity_cols], schema_overrides=schema_electricity)
    df_new_forecast    = pl.from_pandas(forecast_weather[forecast_cols], schema_overrides=schema_forecast)
    df_new_historical  = pl.from_pandas(historical_weather[historical_cols], schema_overrides=schema_historical)
    df_new_target      = pl.from_pandas(revealed_targets[target_cols], schema_overrides=schema_target)
    df_new_data        = pl.from_pandas(revealed_targets[df_data_cols], schema_overrides=schema_data)
    df_client          = pl.concat([df_client, df_new_client]).unique(subset=["county", "is_business", "product_type", "date"], maintain_order=True)
    df_gas             = pl.concat([df_gas, df_new_gas]).unique(subset=["forecast_date"], maintain_order=True)
    df_electricity     = pl.concat([df_electricity, df_new_electricity]).unique(subset=["forecast_date"], maintain_order=True)
    df_forecast        = pl.concat([df_forecast, df_new_forecast]).unique()
    df_historical      = pl.concat([df_historical, df_new_historical]).unique()
    df_target          = pl.concat([df_target, df_new_target]).unique()
    df_data            = pl.concat([df_data, df_new_data]).unique()

    X_test = feature_eng(df_test, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target, working_days)
    X_test = to_pandas(X_test)

    sample_prediction["target"] = model.predict(X_test.drop('datetime', axis=1))
    env.predict(sample_prediction)